<a href="https://colab.research.google.com/github/albivaltzew/NLP_CV_DC_ITMO/blob/main/Unit_4_WordNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [ ]:
with open("Task_4_sample_5.txt", encoding="utf-16") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [ ]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: stock-CD
Ground truth score: 1.31

Path: 0.100 
wup: 0.400 
shortest_path: 9.000

Words: fuck-sex
Ground truth score: 9.44

Path: 0.333 
wup: 0.857 
shortest_path: 2.000


Вычисляем для всех пар несколько оценок

In [ ]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [ ]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.4f" % coef)



wup  Spearman R: 0.6387
path Spearman R: 0.6160
lch Spearman R: 0.6160


In [ ]:
physics = wn.synset('physics.n.01')
typesOfPhysics = list(set([w for s in physics.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
len(typesOfPhysics)

67

In [ ]:
typesOfPhysics

['high-energy_physics',
 'plasma_physics',
 'mechanics',
 'kinematics',
 'magnetohydrodynamics',
 'solid-state_physics',
 'electron_optics',
 'aerodynamics',
 'thermodynamics',
 'Newtonian_mechanics',
 'statistical_mechanics',
 'uranology',
 'rheology',
 'electronics',
 'crystallography',
 'astronautics',
 'electromagnetism',
 'electromagnetics',
 'electrostatics',
 'thermionics',
 'pneumatics',
 'hydrostatics',
 'atomic_physics',
 'quantum_field_theory',
 'astronomy',
 'cosmology',
 'solar_physics',
 'optics',
 'quantum_electrodynamics',
 'wave_mechanics',
 'hydrodynamics',
 'dynamics',
 'cosmogeny',
 'QED',
 'aeronautics',
 'hydrokinetics',
 'quantum_mechanics',
 'nucleonics',
 'microelectronics',
 'aeromechanics',
 'statics',
 'catoptrics',
 'hydraulics',
 'classical_mechanics',
 'biophysics',
 'astrophysics',
 'holography',
 'radio_astronomy',
 'astrodynamics',
 'thermostatics',
 'thermodynamics_of_equilibrium',
 'astrometry',
 'cosmogony',
 'ballistics',
 'particle_physics',
 'QCD

In [ ]:
physics = wn.synset('physics.n.01')
physics.hyponyms()

[Synset('aeronautics.n.01'),
 Synset('astronomy.n.01'),
 Synset('biophysics.n.01'),
 Synset('cryogenics.n.01'),
 Synset('crystallography.n.01'),
 Synset('electromagnetism.n.02'),
 Synset('electronics.n.01'),
 Synset('electrostatics.n.01'),
 Synset('mechanics.n.01'),
 Synset('nuclear_physics.n.01'),
 Synset('optics.n.01'),
 Synset('particle_physics.n.01'),
 Synset('plasma_physics.n.01'),
 Synset('quantum_physics.n.01'),
 Synset('rheology.n.01'),
 Synset('solid-state_physics.n.01'),
 Synset('statistical_mechanics.n.01'),
 Synset('thermodynamics.n.01')]

In [ ]:
physics.hyponyms()[0].name()

'aeronautics.n.01'

In [ ]:
len(physics.hyponyms())

18